## Setup Packages

In [1]:
# reload the modules
%load_ext autoreload
%autoreload 2
from utils import fetch_fi2010
import plotly.express as px
import numpy as np
from hmmlearn import hmm
import plotly.graph_objects as go
import pandas as pd
from utils import distribution_stats,test_distribution,fit_best_HMM,fit_HMM,plot_states,plot_states_shades,detect_DC
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

## Data Preparation

In [2]:
df_fi2010 = fetch_fi2010()    

In [3]:
df_fi2010.head()

,STOCK,DAY,PRICE_ASK_0,PRICE_ASK_1,PRICE_ASK_2,PRICE_ASK_3,PRICE_ASK_4,PRICE_ASK_5,PRICE_ASK_6,PRICE_ASK_7,...,VOLUME_BID_5,VOLUME_BID_6,VOLUME_BID_7,VOLUME_BID_8,VOLUME_BID_9,LABEL_1TICK,LABEL_2TICK,LABEL_3TICK,LABEL_5TICK,LABEL_10TICK
0,1,1,2615,2618,2619,2620,2621,2623,2625,2626,...,100,143,134,123,128,0,0,0,0,0
1,1,1,2615,2619,2620,2621,2625,2626,2629,2633,...,159,100,143,134,123,0,0,0,0,0
2,1,1,2614,2615,2617,2619,2620,2621,2625,2626,...,159,100,143,134,123,-1,-1,0,0,0
3,1,1,2614,2617,2619,2620,2621,2625,2629,2633,...,100,143,134,123,128,0,0,-1,0,0
4,1,1,2614,2617,2619,2620,2621,2625,2629,2633,...,100,143,134,123,128,1,1,1,0,0


In [4]:
df_fi2010.describe()

,STOCK,DAY,PRICE_ASK_0,PRICE_ASK_1,PRICE_ASK_2,PRICE_ASK_3,PRICE_ASK_4,PRICE_ASK_5,PRICE_ASK_6,PRICE_ASK_7,...,VOLUME_BID_5,VOLUME_BID_6,VOLUME_BID_7,VOLUME_BID_8,VOLUME_BID_9,LABEL_1TICK,LABEL_2TICK,LABEL_3TICK,LABEL_5TICK,LABEL_10TICK
count,272941.000000,272941.000000,272941.000000,272941.000000,272941.000000,272941.000000,272941.000000,272941.000000,272941.000000,272941.000000,...,272941.000000,272941.000000,272941.000000,272941.000000,272941.000000,272941.000000,272941.000000,272941.000000,272941.000000,272941.000000
mean,3.544114,5.796187,1760.879087,1762.067667,1763.213086,1764.344525,1765.491634,1766.712242,1768.034681,1769.447152,...,2583.846395,2359.217157,2342.672852,2554.313844,2966.451054,0.001429,0.002440,0.004679,0.006338,0.008925
std,1.225020,2.864668,742.141907,742.406185,742.653852,742.864465,743.074888,743.332344,743.643167,744.040515,...,2544.857222,2496.627949,2973.216188,3854.109729,5017.384342,0.610031,0.690758,0.741689,0.808032,0.884811
min,1.000000,1.000000,1203.000000,1204.000000,1205.000000,1206.000000,1207.000000,1208.000000,1209.000000,1210.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,3.000000,4.000000,1272.000000,1273.000000,1274.000000,1275.000000,1276.000000,1277.000000,1279.000000,1280.000000,...,1000.000000,728.000000,619.000000,600.000000,553.000000,0.000000,0.000000,-1.000000,-1.000000,-1.000000
50%,4.000000,6.000000,1319.000000,1320.000000,1321.000000,1322.000000,1323.000000,1324.000000,1325.000000,1326.000000,...,1913.000000,1643.000000,1484.000000,1422.000000,1383.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,5.000000,9.000000,1768.000000,1769.000000,1770.000000,1771.000000,1772.000000,1773.000000,1774.000000,1775.000000,...,3521.000000,3204.000000,2998.000000,3028.000000,3224.000000,0.000000,0.000000,1.000000,1.000000,1.000000
max,5.000000,10.000000,3692.000000,3697.000000,3698.000000,3699.000000,3700.000000,3701.000000,3705.000000,3710.000000,...,33145.000000,36224.000000,42299.000000,40351.000000,50863.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [5]:
df_fi2010.columns

Index(['STOCK', 'DAY', 'PRICE_ASK_0', 'PRICE_ASK_1', 'PRICE_ASK_2',
       'PRICE_ASK_3', 'PRICE_ASK_4', 'PRICE_ASK_5', 'PRICE_ASK_6',
       'PRICE_ASK_7', 'PRICE_ASK_8', 'PRICE_ASK_9', 'PRICE_BID_0',
       'PRICE_BID_1', 'PRICE_BID_2', 'PRICE_BID_3', 'PRICE_BID_4',
       'PRICE_BID_5', 'PRICE_BID_6', 'PRICE_BID_7', 'PRICE_BID_8',
       'PRICE_BID_9', 'VOLUME_ASK_0', 'VOLUME_ASK_1', 'VOLUME_ASK_2',
       'VOLUME_ASK_3', 'VOLUME_ASK_4', 'VOLUME_ASK_5', 'VOLUME_ASK_6',
       'VOLUME_ASK_7', 'VOLUME_ASK_8', 'VOLUME_ASK_9', 'VOLUME_BID_0',
       'VOLUME_BID_1', 'VOLUME_BID_2', 'VOLUME_BID_3', 'VOLUME_BID_4',
       'VOLUME_BID_5', 'VOLUME_BID_6', 'VOLUME_BID_7', 'VOLUME_BID_8',
       'VOLUME_BID_9', 'LABEL_1TICK', 'LABEL_2TICK', 'LABEL_3TICK',
       'LABEL_5TICK', 'LABEL_10TICK'],
      dtype='object')

Get mid price TOB and returns of the mid price of the first stock

In [6]:
selected_data= df_fi2010.loc[(df_fi2010.DAY<=3) & (df_fi2010.STOCK==2)]
selected_data["MID_PRICE"] = selected_data[["PRICE_ASK_0","PRICE_BID_0"]].mean(axis=1)
selected_data["RETURN_MID_PRICE"] = selected_data["MID_PRICE"].pct_change().fillna(0)
len(selected_data)

C:\Users\dfmic\AppData\Local\Temp\ipykernel_1268\3077512736.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data["MID_PRICE"] = selected_data[["PRICE_ASK_0","PRICE_BID_0"]].mean(axis=1)
C:\Users\dfmic\AppData\Local\Temp\ipykernel_1268\3077512736.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data["RETURN_MID_PRICE"] = selected_data["MID_PRICE"].pct_change().fillna(0)


6705

In [7]:
px.line(selected_data, y=["PRICE_ASK_0","PRICE_BID_0","MID_PRICE"])

In [8]:
px.line(selected_data, y=["RETURN_MID_PRICE"])

In [9]:
px.histogram(selected_data, y=["RETURN_MID_PRICE"], marginal= "violin")

In [10]:
R=[abs(r)*10000 for r in detect_DC(selected_data["MID_PRICE"].to_list())]
print(len(R))

640


In [11]:
px.line(R)

information on returns

In [12]:
distribution_stats(selected_data["RETURN_MID_PRICE"])

Stylised fact of the retrun distribution
--------------------------------------------------
Length of the return series: (6705,)
Mean: 0.0005523431336806113 %
Standard Deviation: 0.026821503519591876 %
Skew: 1.8922603217253589
Kurtosis: 70.14261715305837


In [13]:
test_distribution(selected_data["RETURN_MID_PRICE"], 'mean')
test_distribution(selected_data["RETURN_MID_PRICE"], 'normal')


p = 0.0917917
The null hypothesis cannot be rejected
p = 0
The null hypothesis can be rejected


In [14]:
train, test = train_test_split(R, test_size=0.2,shuffle=False)
# train, test = train_test_split(selected_data["RETURN_MID_PRICE"]*100, test_size=0.2)
relabeled_states, mus, sigmas, transmat, post_prob, model = fit_best_HMM(train, test,3,iterations=1)
df_post_prob= pd.DataFrame(post_prob)

In [15]:
plot_states(train, relabeled_states)

In [16]:
plot_states_shades(train,df_post_prob)

In [343]:
test_vals = np.expand_dims(test, 1)
test_vals = np.reshape(test_vals,[len(test),1])

prediction = model.predict(test_vals)

In [344]:
plot_states(test, prediction)

In [345]:
mod_hamilton = sm.tsa.MarkovAutoregression(train, k_regimes=3, order = 4, trend="n", switching_ar = False, switching_variance = True)
res_hamilton = mod_hamilton.fit()
res_hamilton.summary()

In [ ]:
plot_states_shades(train,res_hamilton.smoothed_marginal_probabilities)

AttributeError: 'numpy.ndarray' object has no attribute 'iloc'

In [ ]:
plot_states(train**(0.5)/100, res_hamilton.smoothed_marginal_probabilities.idxmax(axis=1))

In [ ]:
prediction2 = res_hamilton.predict(test_vals)

In [ ]:
prediction2